In [1]:
import cirq
import numpy as np

# The Portfolio Rebalancing Optimization using QAOA with soft constraints

In this notebook, we will apply the QAOA algorithm to solve the portfolio optimization problem using a soft constraint


$$
C(z) = \sum_i \lambda r_i z_i - (1-\lambda)\sum_{ij} \sigma_{ij} z_i z_j
$$

$$
C_R(z) = \sum_i T (1- \delta())
$$


The soft constraint that we impose will be given 

$$
H = C(z)+A(D-\sum_i z_i)^2
$$